In [46]:
# Settings

# General settings
data_folder = "/home/andrea/Projects/WMO_Volta/data/data_dynamic/discharge/" #"/home/andrea/Desktop/Test/series/" 
start_analysis = "2018-01-01 00:00"
end_analysis = "2020-12-31 23:00"
freq="D"

# Hmc settings
domain = "volta"
hmc_output = "/home/andrea/Projects/WMO_Volta/test/imerg/hmc.hydrograph.txt"
hmc_static_gridded = "/home/andrea/Projects/WMO_Volta/data/data_static/volta_imerg/gridded/"
hmc_static_point = "/home/andrea/Projects/WMO_Volta/data/data_static/volta_imerg/point/"

# Data
et_series = "/home/andrea/Projects/WMO_Volta/test/imerg/average_ETCum.txt"
rainfall_series = "/home/andrea/Projects/WMO_Volta/test/imerg/average_rainfall_mm_h.txt"

In [47]:
# Imports
%matplotlib inline

from ipywidgets import interactive,HBox
import pandas as pd
import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
from pysheds.grid import Grid

import ipywidgets as widgets
from IPython.display import display, HTML
import os, glob
import datetime as dt
import matplotlib.lines as mlines

# Define custom functions
def create_df(choices):
    df = pd.DataFrame(index=pd.date_range(start_time,end_time,freq=freq), columns=choices)
    for name in choices:
        series = pd.read_csv(os.path.join(data_folder, name + ".csv"), index_col=0, header=0, parse_dates=True)
        df[name] = series.reindex(pd.date_range(start_time,end_time,freq=freq), method=None)
    return df

def multiplot(widg):
    choices = widg['new']
    df = create_df(choices)
    data = df.loc[:, choices] if choices else df
    output.clear_output(wait=True)
    with output:
        ax = data.plot(figsize=(10,7))
        plt.show()
        
def combinedplot(widg):
    choices = widg['new']
    data_et = series["ET"].loc[:, choices] if choices else series
    data_rain = series["rain"].loc[:, choices] if choices else series
    data_qMod = series["discharge_mod"].loc[:, choices] if choices else series
    data_qObs = series["discharge_obs"].loc[:, choices] if choices else series
    output2.clear_output(wait=True)
    with output2:
        ax = data_rain.plot(figsize=(15,7), color='c') 
        plt.ylim(bottom=0)
        plt.ylabel("mm of rain")
        axx = ax.twinx()
        ax1 = data_et.plot(figsize=(15,7), color='g')
        #ax1 = plt.plot(data_qMod.index, data_et.values)
        ax2 = data_qMod.plot(figsize=(15,7), color='b')
        ax3 = data_qObs.plot(figsize=(15,7), color='r', style='.')
        plt.ylim(bottom=0)
        plt.ylabel("mm")
        cyan_line = mlines.Line2D([], [], color='cyan', label='rain')
        green_line = mlines.Line2D([], [], color='green', label='et')
        blue_line = mlines.Line2D([], [], color='blue', label='mod_dis')
        red_line = mlines.Line2D([], [], color='red', label='obs_dis')
        
        plt.legend(handles=[cyan_line, green_line, blue_line, red_line])
        plt.show()

def read_discharge_hmc(file='', col_names=None):
    custom_date_parser = lambda x: dt.datetime.strptime(x, "%Y%m%d%H%M")
    hmc_discharge_df = pd.read_csv(file, header=None, delimiter=r"\s+", parse_dates=[0], index_col=[0], date_parser=custom_date_parser)
    hmc_discharge_df.columns = col_names
    return hmc_discharge_df

In [48]:
# Read static inputs

# Sections
sections = tabular = pd.read_csv(os.path.join(hmc_static_point, domain + ".info_section.txt"), sep="\s+", header=None)
rHMC, cHMC, basin_name, section_name = tabular.values[:,0], tabular.values[:,1], tabular.values[:,2], tabular.values[:,3]

# Pointers
grid = Grid.from_ascii(os.path.join(hmc_static_gridded, domain + ".pnt.txt"))
pnt = grid.read_ascii(os.path.join(hmc_static_gridded, domain + ".pnt.txt"), dtype=np.int8)
areacell = grid.read_ascii(os.path.join(hmc_static_gridded, domain + ".areacell.txt"))

In [49]:
# Compute basin cell area
basin_area = pd.DataFrame(index=section_name, columns=["Area (m2)"])
dirmap_HMC = (8, 9, 6, 3, 2, 1, 4, 7)

for ix, iy, basin, name in zip(cHMC, rHMC, basin_name, section_name):
        basin = grid.catchment(fdir=pnt, x=ix-1, y=iy-1, dirmap=dirmap_HMC, xytype='index')
        mask = np.where(basin>0, 1, np.nan)
        basin_area.loc[name, "Area (m2)"] = np.nansum(mask*areacell).astype("float32")

In [50]:
# Read hmc output
mod_out = read_discharge_hmc(hmc_output, section_name)

In [51]:
# Set timing
start_time = dt.datetime.strptime(start_analysis, "%Y-%m-%d %H:%M")
end_time = dt.datetime.strptime(end_analysis, "%Y-%m-%d %H:%M")

In [52]:
# Read datasets
series = {}

series["ET"] = pd.read_csv(et_series, index_col=0, header=0, parse_dates=True)[start_time:end_time]
series["rain"] = pd.read_csv(rainfall_series, index_col=0, header=0, parse_dates=True)[start_time:end_time]
dis_out = mod_out[start_time:end_time]

missing_data = [i for i in section_name if not os.path.isfile(os.path.join(data_folder, i + ".csv"))]
display("WARNING! Data for sections " + ", ".join(missing_data) + " are missing!")
dis_in = create_df([i for i in section_name if i not in missing_data])

'WARNING! Data for sections Arly_doudouro, Atchangbade, Bagre_aval_au_pont, Bassar, Bayé, Borgou, Brouffou, Ekumdipe, Gbanlou-bineda_Kopingue, Gorée, Gougoulo_Doropo, Kara-kpessidè, Koulda_Pouon, Koumongou, Kounou, LeryNord, Naboulgou, Nangodi, Nano, Nebou, Niaogho, Nnaboupi, Noberé, Noumbiel, Pletou, Porga, Pouda, Prang, Rte-SokodeBassar, Samboali, Sanogo, Tagou, Tampelga, Titira, Tourouba, Volta_Vonkoro, Wiasi, Yilou are missing!'

In [53]:
# Convert all series to mm/day
series["rain"] = series["rain"].resample("D").sum()
series["discharge_mod"] = pd.DataFrame(index = series["rain"].index, columns = dis_out.columns)
series["discharge_obs"] = pd.DataFrame(index = series["rain"].index, columns = dis_in.columns)
series["availability"] = pd.DataFrame(index = series["rain"].index, columns = dis_in.columns)

for name in dis_out.columns:
    series["discharge_mod"][name] = (dis_out[name] * (1000*3600)/float(basin_area.loc[name, "Area (m2)"])).resample("D").sum()

for name in series["discharge_obs"].columns:
    series["discharge_obs"][name] = (dis_in[name] * (1000*3600*24)/float(basin_area.loc[name, "Area (m2)"]))
    series["availability"][name] = dis_in[name].resample("D").count()

In [54]:
series["ET"] = pd.DataFrame(series["ET"].values, index=series["rain"].index, columns=series["ET"].columns)

In [55]:
series_annual = {}

for stype in series.keys():
    series_annual[stype] = series[stype].resample("Y").sum()
    index = pd.Index([str(y) + "_" + stype for y in np.unique(series_annual[stype].index.year)])
    series_annual[stype].set_index(index, inplace=True)

In [56]:
out_table = pd.concat([series_annual[i].T for i in series_annual.keys()], axis=1)
out_table = out_table.reindex(sorted(out_table.columns), axis=1)

display(out_table)
out_table.to_csv("/home/andrea/Desktop/summary_balance_old.csv")

,2018_ET,2018_availability,2018_discharge_mod,2018_discharge_obs,2018_rain,2019_ET,2019_availability,2019_discharge_mod,2019_discharge_obs,2019_rain,2020_ET,2020_availability,2020_discharge_mod,2020_discharge_obs,2020_rain
Arly_doudouro,733.065722,NaN,48.886370,NaN,790.921938,841.034615,NaN,100.306559,NaN,970.648474,800.285171,NaN,153.882048,NaN,956.784302
Atchangbade,811.038909,NaN,238.245471,NaN,1160.264104,802.449130,NaN,267.650344,NaN,1226.638043,845.255631,NaN,449.505548,NaN,1469.368808
Badara,889.180624,290.0,302.663595,79.559732,1242.660414,933.172536,299.0,337.360675,49.561959,1314.401025,859.530167,312.0,228.761780,66.444979,1082.414506
Bagre_aval_au_pont,767.682685,NaN,66.332360,NaN,899.876780,816.003944,NaN,38.355597,NaN,866.015421,753.583611,NaN,187.740325,NaN,945.909912
Bamboi,813.256358,365.0,112.240279,73.350016,984.128883,847.316021,365.0,201.639002,89.788584,1065.681059,810.165314,0.0,102.293947,0.000000,892.398340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yilou,770.829070,NaN,110.112937,NaN,948.616621,769.373644,NaN,2.680817,NaN,765.908095,709.396958,NaN,100.752403,NaN,817.367669
Ziou,770.443603,359.0,143.328266,56.807634,953.586745,869.798983,363.0,155.587073,44.856950,1051.821795,796.516456,274.0,253.679326,49.546870,1037.767736
Gbanlou-bineda_Kopingue,NaN,NaN,89.535083,NaN,NaN,NaN,NaN,242.893022,NaN,NaN,NaN,NaN,39.588051,NaN,NaN
Kara-kpessidè,NaN,NaN,358.191998,NaN,NaN,NaN,NaN,401.316594,NaN,NaN,NaN,NaN,625.186205,NaN,NaN


In [57]:
# Generate list
selector2 = widgets.Dropdown(
options=series_annual["ET"].T.index,
value=series_annual["ET"].T.index[0])

output2 = widgets.Output()

# Set layout and display
form_item_layout = widgets.Layout(
    display='flex',
    justify_content='space-between'
)
display(widgets.VBox([selector2, output2], layout=form_item_layout))

# Re-generate multiplot
selector2.observe(combinedplot, names='value')